<a href="https://colab.research.google.com/github/GuilhermeVCCdaSilva/ProjetoBD/blob/main/ProjetoBigData_GuilhermeSilvaMarineFournieGon%C3%A7aloAlves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://www.estbarreiro.ips.pt/Content/images/logo-ESTB.png" align="right" alt="EsT Logo circle logo" style="height: 200px; width:300px;"/>

# Projeto Final Big Data 
* __Guilherme Silva__
* __Marine Emilie Fournier__
* __Gonçalo Alves__

In [1]:
# Installing required packages
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Creating a SparkSession
Uma **SparkSession** é o ponto de entrada principal para trabalhar com dados no Spark. Ele é responsável por gerenciar a configuração do Spark, criar RDDs (Resilient Distributed Datasets) e DataFrames, registrar tabelas temporárias e gerenciar o conjunto de recursos de cluster que estão sendo usados.

A SparkSession é a classe principal do Spark SQL, que é o módulo do Spark que permite trabalhar com dados estruturados e relacionais. Ele permite que você execute consultas SQL, manipule DataFrames e extraia metadados. Ele também oferece acesso às bibliotecas de processamento de fluxo de dados (como o DataFrame API e o SQL) e às bibliotecas de aprendizado de máquina (como o MLlib) do Spark.

Uma SparkSession é criada usando o SparkSession.builder. É possível configurar a sessão com várias opções, como o nome do aplicativo, o endereço do cluster, o número de núcleos para usar e a quantidade de memória. Além disso, é possível usar a SparkSession para gerenciar configurações globais, como as configurações de memória e de arquivo de log.

.config("spark.memory.offHeap.enabled","true") é uma configuração utilizada para habilitar o uso de memória fora da heap (off-heap) no Spark. Quando essa configuração é definida como "true", o Spark pode usar memória fora do heap do JVM para armazenar os dados e realizar operações. Isso pode ser útil quando o heap do JVM não é suficiente para armazenar todos os dados, permitindo que o Spark use mais memória do que o heap para melhorar o desempenho das operações.

.config("spark.memory.offHeap.size","10g") é uma configuração utilizada para definir a quantidade de memória fora da heap (off-heap) que o Spark pode usar. O valor "10g" indica que o Spark pode usar até 10 gigabytes de memória fora da heap. Esse valor pode ser ajustado de acordo com o tamanho dos dados e as necessidades do seu aplicativo.

In [2]:
from pyspark.sql import SparkSession
import findspark

findspark.init()

spark = SparkSession.builder.appName("Pyspark Tutorial")\
                    .config("spark.memory.offHeap.enabled","true")\
                    .config("spark.memory.offHeap.size","10g")\
                    .getOrCreate()
spark                    

Carregamento dos dados do DataSet
O dataset pode ser obtido em https://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work

In [3]:
!curl https://raw.githubusercontent.com/GuilhermeVCCdaSilva/ProjetoBD/main/Absenteeism_at_work.csv >> Absenteeism_at_work.csv
df = spark.read.csv('Absenteeism_at_work.csv',header=True,sep=";")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45232  100 45232    0     0   552k      0 --:--:-- --:--:-- --:--:--  552k


In [4]:
df.show(10)

+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+
| ID|Reason for absence|Month of absence|Day of the week|Seasons|Transportation expense|Distance from Residence to Work|Service time|Age|Work load Average/day |Hit target|Disciplinary failure|Education|Son|Social drinker|Social smoker|Pet|Weight|Height|Body mass index|Absenteeism time in hours|
+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+
| 11|                26|               7|              3|      1|                   289|                        

#Este DataSet contém várias colunas tais como:

1.  ID: Identificador único para cada registo.
2.    Razão para a ausência: Razão para a ausência do funcionário no trabalho.
3.    Mês da ausência: O mês em que o funcionário estava ausente.
4.    Dia da semana: O dia da semana em que o funcionário estava ausente.
5.    Estações: A estação do ano em que o funcionário estava ausente.
6.    Despesas de transporte: As despesas de transporte incorridas pelo funcionário ao viajar para o trabalho.
7.    Distância da residência para o trabalho: A distância entre a residência do funcionário e o local de trabalho.
8.    Tempo de serviço: O período de tempo em que o funcionário está a trabalhar na empresa.
9.    Idade: A idade do funcionário.
10.    Carga de trabalho média/dia: A carga de trabalho média do funcionário por dia.
11.    Alvo atingido: Se o funcionário atingiu ou não o alvo
12.    Falha disciplinar: Se o funcionário cometeu alguma falha disciplinar ou não.
12.    Educação: O nível de educação do funcionário.
14.    Filhos: O número de filhos que o funcionário tem.
15.    Bebedor social: Se o funcionário bebe socialmente ou não.
16.    Fumador social: Se o funcionário fuma socialmente ou não.
17.    Animal de estimação: Se o funcionário tem algum animal de estimação ou não.
18.    Peso: O peso do funcionário.
19.    Altura: A altura do funcionário.
20.    Índice de massa corporal: O índice de massa corporal do funcionário.
21.    Tempo de ausência em horas: O número de horas que o funcionário ficou ausente.

In [5]:
df = df.drop("id")

#Exploratory Data Analysis

- Check the number of rows and columns in the dataset

In [6]:
print("Number of rows:", df.count())
print("Number of columns:", len(df.columns))

Number of rows: 7409
Number of columns: 20


- Check the number of missing values in each column

In [7]:
# create a list to store the number of missing values in each column
missing_values = []

# iterate through the columns in the dataset
for col in df.columns:
    missing_values.append((col, df.filter(df[col].isNull()).count()))

# print the missing values
for col, val in missing_values:
    if val == 0:
        print("{} : No missing values".format(col))
    else:
        print("{} : {} missing values".format(col, val))

Reason for absence : No missing values
Month of absence : No missing values
Day of the week : No missing values
Seasons : No missing values
Transportation expense : No missing values
Distance from Residence to Work : No missing values
Service time : No missing values
Age : No missing values
Work load Average/day  : No missing values
Hit target : No missing values
Disciplinary failure : No missing values
Education : No missing values
Son : No missing values
Social drinker : No missing values
Social smoker : No missing values
Pet : No missing values
Weight : No missing values
Height : No missing values
Body mass index : No missing values
Absenteeism time in hours : No missing values


- check the data types of the columns

In [8]:
df.dtypes

[('Reason for absence', 'string'),
 ('Month of absence', 'string'),
 ('Day of the week', 'string'),
 ('Seasons', 'string'),
 ('Transportation expense', 'string'),
 ('Distance from Residence to Work', 'string'),
 ('Service time', 'string'),
 ('Age', 'string'),
 ('Work load Average/day ', 'string'),
 ('Hit target', 'string'),
 ('Disciplinary failure', 'string'),
 ('Education', 'string'),
 ('Son', 'string'),
 ('Social drinker', 'string'),
 ('Social smoker', 'string'),
 ('Pet', 'string'),
 ('Weight', 'string'),
 ('Height', 'string'),
 ('Body mass index', 'string'),
 ('Absenteeism time in hours', 'string')]

- Convert data type string to double

In [9]:
from pyspark.sql.functions import col
for column in df.columns:
    df = df.withColumn(column, col(column).cast("double"))

In [10]:
df.dtypes

[('Reason for absence', 'double'),
 ('Month of absence', 'double'),
 ('Day of the week', 'double'),
 ('Seasons', 'double'),
 ('Transportation expense', 'double'),
 ('Distance from Residence to Work', 'double'),
 ('Service time', 'double'),
 ('Age', 'double'),
 ('Work load Average/day ', 'double'),
 ('Hit target', 'double'),
 ('Disciplinary failure', 'double'),
 ('Education', 'double'),
 ('Son', 'double'),
 ('Social drinker', 'double'),
 ('Social smoker', 'double'),
 ('Pet', 'double'),
 ('Weight', 'double'),
 ('Height', 'double'),
 ('Body mass index', 'double'),
 ('Absenteeism time in hours', 'double')]

In [11]:
from pyspark.sql.functions import when
from pyspark.sql.functions import array
from pyspark.sql.functions import udf
# Create the mapping dictionary
mapping_dict = {
    0: "CID - Patient follow-up",
    1: "CID - Medical consultation",
    2: "CID - Blood donation",
    3: "CID - Laboratory examination",
    4: "CID - Unjustified absence",
    5: "CID - Physiotherapy",
    6: "CID - Dental consultation",
    7: "Certain infectious and parasitic diseases",
    8: "Neoplasms",
    9: "Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism",
    10: "Endocrine, nutritional and metabolic diseases",
    11: "Mental and behavioural disorders",
    12: "Diseases of the nervous system",
    13: "Diseases of the eye and adnexa",
    14: "Diseases of the ear and mastoid process",
    15: "Diseases of the circulatory system",
    16: "Diseases of the respiratory system",
    17: "Diseases of the digestive system",
    18: "Diseases of the skin and subcutaneous tissue",
    19: "Diseases of the musculoskeletal system and connective tissue",
    20: "Diseases of the genitourinary system",
    21: "Pregnancy, childbirth and the puerperium",
    22: "Certain conditions originating in the perinatal period",
    23: "Congenital malformations, deformations and chromosomal abnormalities",
    24: "Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified",
    25: "Injury, poisoning and certain other consequences of external causes",
    26: "External causes of morbidity and mortality",
    27: "Factors influencing health status and contact with health services"
}

# Create the user-defined function
map_reason = udf(lambda x: mapping_dict.get(x))

# Apply the user-defined function to the dataframe
df = df.withColumn("Reason for absence", map_reason(col("Reason for absence")))


In [12]:
df.select("Reason for absence").show()

+--------------------+
|  Reason for absence|
+--------------------+
|External causes o...|
|CID - Patient fol...|
|Congenital malfor...|
|Certain infectiou...|
|Congenital malfor...|
|Congenital malfor...|
|Certain condition...|
|Congenital malfor...|
|Diseases of the m...|
|Certain condition...|
|CID - Medical con...|
|CID - Medical con...|
|Mental and behavi...|
|Mental and behavi...|
|Congenital malfor...|
|Diseases of the e...|
|Congenital malfor...|
|Pregnancy, childb...|
|Mental and behavi...|
|Congenital malfor...|
+--------------------+
only showing top 20 rows



In [13]:
df.show(10)

+--------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+----+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+
|  Reason for absence|Month of absence|Day of the week|Seasons|Transportation expense|Distance from Residence to Work|Service time| Age|Work load Average/day |Hit target|Disciplinary failure|Education|Son|Social drinker|Social smoker|Pet|Weight|Height|Body mass index|Absenteeism time in hours|
+--------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+----+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+
|External causes o...|             7.0|            3.0|    1.0|                 289.0|                           36

In [18]:
df.dtypes

[('Reason for absence', 'string'),
 ('Month of absence', 'double'),
 ('Day of the week', 'double'),
 ('Seasons', 'double'),
 ('Transportation expense', 'double'),
 ('Distance from Residence to Work', 'double'),
 ('Service time', 'double'),
 ('Age', 'double'),
 ('Work load Average/day ', 'double'),
 ('Hit target', 'double'),
 ('Disciplinary failure', 'double'),
 ('Education', 'double'),
 ('Son', 'double'),
 ('Social drinker', 'double'),
 ('Social smoker', 'double'),
 ('Pet', 'double'),
 ('Weight', 'double'),
 ('Height', 'double'),
 ('Body mass index', 'double'),
 ('Absenteeism time in hours', 'double')]

- Check the statistics of numerical columns

In [14]:
df.describe().toPandas()

,summary,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,count,6280,7400,7400,7400,7400,7400,7400,7400,7400,...,7400,7400,7400,7400,7400,7400,7400,7400,7400,7400
1,mean,None,6.324324324324325,3.9148648648648647,2.5445945945945945,221.32972972972973,29.63108108108108,12.554054054054054,36.45,271.49023513513544,...,0.05405405405405406,1.2918918918918918,1.018918918918919,0.5675675675675675,0.07297297297297298,0.745945945945946,79.03513513513514,172.11486486486487,26.677027027027027,6.924324324324324
2,stddev,None,3.4341963796261354,1.4208097978982286,1.111154649540044,66.91149116264964,14.827762100288005,4.382205758699573,6.474830932109758,39.03435419279272,...,0.22613961171543107,0.6728284600917894,1.0978207258816615,0.4954470659280054,0.26010970961220287,1.3174562954681877,12.875372679394976,6.0313229883202135,4.282845059644217,13.322887850259045
3,min,CID - Blood donation,0.0,2.0,1.0,118.0,5.0,1.0,27.0,205.917,...,0.0,1.0,0.0,0.0,0.0,0.0,56.0,163.0,19.0,0.0
4,max,"Symptoms, signs and abnormal clinical and labo...",12.0,6.0,4.0,388.0,52.0,29.0,58.0,378.884,...,1.0,4.0,4.0,1.0,1.0,8.0,108.0,196.0,38.0,120.0


- Check the correlation between columns

In [15]:
corr = df.stat.corr("Month of absence", "Transportation expense")
print(corr)

0.14367578966445316


- Check the number of unique values in each column

In [16]:
for col in df.columns:
    print("Unique values in column '{}':".format(col), df.select(col).distinct().count())

Unique values in column 'Reason for absence': 28
Unique values in column 'Month of absence': 14
Unique values in column 'Day of the week': 6
Unique values in column 'Seasons': 5
Unique values in column 'Transportation expense': 25
Unique values in column 'Distance from Residence to Work': 26
Unique values in column 'Service time': 19
Unique values in column 'Age': 23
Unique values in column 'Work load Average/day ': 39
Unique values in column 'Hit target': 14
Unique values in column 'Disciplinary failure': 3
Unique values in column 'Education': 5
Unique values in column 'Son': 6
Unique values in column 'Social drinker': 3
Unique values in column 'Social smoker': 3
Unique values in column 'Pet': 7
Unique values in column 'Weight': 27
Unique values in column 'Height': 15
Unique values in column 'Body mass index': 18
Unique values in column 'Absenteeism time in hours': 20


- check the distribution of each column

In [17]:
for col in df.columns:
    df.select(col).groupBy(col).count().show()

+--------------------+-----+
|  Reason for absence|count|
+--------------------+-----+
|Congenital malfor...| 1490|
|Symptoms, signs a...|   30|
|CID - Dental cons...|   80|
|CID - Unjustified...|   20|
|                null| 1129|
|Diseases of the e...|  550|
|CID - Laboratory ...|   10|
|CID - Blood donation|   10|
|Injury, poisoning...|  310|
|           Neoplasms|   60|
|CID - Patient fol...|  430|
|Pregnancy, childb...|   60|
|Endocrine, nutrit...|  250|
|Diseases of the b...|   40|
|Diseases of the c...|   20|
| CID - Physiotherapy|   30|
|Diseases of the m...|  400|
|Diseases of the e...|  190|
|Certain condition...|  380|
|Diseases of the s...|  210|
+--------------------+-----+
only showing top 20 rows

+----------------+-----+
|Month of absence|count|
+----------------+-----+
|             8.0|  540|
|             0.0|   30|
|             7.0|  670|
|            null|    9|
|             1.0|  500|
|             4.0|  530|
|            11.0|  630|
|             3.0|  870|
|  